In [1]:
import torch 
from torch import nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
import os,PIL,pathlib
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

#设置GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
data_dir = './data/bird_photos'
data_dir = pathlib.Path(data_dir)

In [3]:
image_count = len(list(data_dir.glob('*/*')))
image_count # 这里输出的是图片的总数

566

In [4]:
data_dir = './data/bird_photos'
data_transform = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor()
])

In [5]:
total_data = datasets.ImageFolder(data_dir,transform=data_transform)
total_data

Dataset ImageFolder
    Number of datapoints: 566
    Root location: ./data/bird_photos
    StandardTransform
Transform: Compose(
               Resize(size=[224, 224], interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )

In [6]:
train_size = int(0.8*len(total_data))
test_size = len(total_data) - train_size
train_data,test_data = torch.utils.data.random_split(total_data,[train_size,test_size])

In [7]:
train_size,test_size

(452, 114)

In [8]:
batch_size = 8
train_dl = torch.utils.data.DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_dl = torch.utils.data.DataLoader(test_data,batch_size=batch_size)

In [9]:
class Block(nn.Module):
    def __init__(self, in_channel, mid_channel, out_channel, dense_channel, stride, groups, is_shortcut=False):
        # in_channel,是输入通道数,mid_channel是中间经历的通道数，out_channels是经过一次板块之后的输出通道数。
        # dense_channels设置这个参数的原因就是一边进行着resnet方式的卷积运算，另一边也同时进行着dense的卷积计算，之后特征图融合形成新的特征图
        super().__init__()
        self.is_shortcut = is_shortcut
        self.out_channel = out_channel
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channel, mid_channel, kernel_size=1, bias=False),
            nn.BatchNorm2d(mid_channel),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(mid_channel, mid_channel, kernel_size=3, stride=stride, padding=1, groups=groups, bias=False),
            nn.BatchNorm2d(mid_channel),
            nn.ReLU()
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(mid_channel, out_channel+dense_channel, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channel+dense_channel)
        )
        if self.is_shortcut:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channel, out_channel+dense_channel, kernel_size=3, padding=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channel+dense_channel)
            )
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        a = x
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        if self.is_shortcut:
            a = self.shortcut(a)
        d = self.out_channel
        x = torch.cat([a[:,:d,:,:] + x[:,:d,:,:], a[:,d:,:,:], x[:,d:,:,:]], dim=1)
        x = self.relu(x)
        return x


class DPN(nn.Module):
    def __init__(self, cfg):
        super(DPN, self).__init__()
        self.group = cfg['group']
        self.in_channel = cfg['in_channel']
        mid_channels = cfg['mid_channels']
        out_channels = cfg['out_channels']
        dense_channels = cfg['dense_channels']
        num = cfg['num']
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, self.in_channel, 7, stride=2, padding=3, bias=False, padding_mode='zeros'),
            nn.BatchNorm2d(self.in_channel),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        )
        self.conv2 = self._make_layers(mid_channels[0], out_channels[0], dense_channels[0], num[0], stride=1)
        self.conv3 = self._make_layers(mid_channels[1], out_channels[1], dense_channels[1], num[1], stride=2)
        self.conv4 = self._make_layers(mid_channels[2], out_channels[2], dense_channels[2], num[2], stride=2)
        self.conv5 = self._make_layers(mid_channels[3], out_channels[3], dense_channels[3], num[3], stride=2)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(cfg['out_channels'][3] + (num[3]+1) * cfg['dense_channels'][3], cfg['classes']) # fc层需要计算
    
    def _make_layers(self, mid_channel, out_channel, dense_channel, num, stride=2):
        layers = []
        layers.append(Block(self.in_channel, mid_channel, out_channel, dense_channel, stride=stride, groups=self.group, is_shortcut=True))
        # block_1里面is_shortcut=True就是resnet中的shortcut连接，将浅层的特征进行一次卷积之后与进行三次卷积的特征图相加
        # 后面几次相同的板块is_shortcut=False简单的理解就是一个多次重复的板块，第一次利用就可以满足浅层特征的利用，后面重复的不在需要
        self.in_channel = out_channel + dense_channel*2
        # 由于里面包含dense这种一直在叠加的特征图计算，
        # 所以第一次是2倍的dense_channel,后面每次一都会多出1倍，所以有(i+2)*dense_channel
        for i in range(1, num):
            layers.append(Block(self.in_channel, mid_channel, out_channel, dense_channel, stride=1, groups=self.group))
            self.in_channel = self.in_channel + dense_channel
            #self.in_channel = out_channel + (i+2)*dense_channel
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.pool(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)
        return x


def DPN92(n_class=10):
    cfg = {
        'group': 32,
        'in_channel': 64,
        'mid_channels': (96, 192, 384, 768),
        'out_channels': (256, 512, 1024, 2048),
        'dense_channels': (16, 32, 24, 128),
        'num': (3, 4, 20, 3),
        'classes': (n_class)
    }
    return DPN(cfg)


def DPN98(n_class=10):
    cfg = {
        'group': 40,
        'in_channel': 96,
        'mid_channels': (160, 320, 640, 1280),
        'out_channels': (256, 512, 1024, 2048),
        'dense_channels': (16, 32, 32, 128),
        'num': (3, 6, 20, 3),
        'classes': (n_class)
    }
    return DPN(cfg)

In [10]:
# 定义完成，测试一下
x = torch.randn(2, 3, 224, 224)
model = DPN98(4)
out = model(x)
print('out.shape: ', out.shape)
print(out)

out.shape:  torch.Size([2, 4])
tensor([[ 0.0879, -0.1848, -0.1012,  0.8561],
        [ 0.1401, -0.1126,  0.0492,  0.8067]], grad_fn=<AddmmBackward>)


In [13]:
model.to(device)
# 统计模型参数量以及其他指标
import torchsummary as summary
summary.summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 96, 112, 112]          14,112
       BatchNorm2d-2         [-1, 96, 112, 112]             192
              ReLU-3         [-1, 96, 112, 112]               0
         MaxPool2d-4           [-1, 96, 55, 55]               0
            Conv2d-5          [-1, 160, 55, 55]          15,360
       BatchNorm2d-6          [-1, 160, 55, 55]             320
              ReLU-7          [-1, 160, 55, 55]               0
            Conv2d-8          [-1, 160, 55, 55]           5,760
       BatchNorm2d-9          [-1, 160, 55, 55]             320
             ReLU-10          [-1, 160, 55, 55]               0
           Conv2d-11          [-1, 272, 55, 55]          43,520
      BatchNorm2d-12          [-1, 272, 55, 55]             544
           Conv2d-13          [-1, 272, 55, 55]         235,008
      BatchNorm2d-14          [-1, 272,

In [14]:
# 训练循环
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)  # 训练集的大小
    num_batches = len(dataloader)   # 批次数目, (size/batch_size，向上取整)

    train_loss, train_acc = 0, 0  # 初始化训练损失和正确率
    
    for X, y in dataloader:  # 获取图片及其标签
        X, y = X.to(device), y.to(device)
        
        # 计算预测误差
        pred = model(X)          # 网络输出
        loss = loss_fn(pred, y)  # 计算网络输出和真实值之间的差距，targets为真实值，计算二者差值即为损失
        
        # 反向传播
        optimizer.zero_grad()  # grad属性归零
        loss.backward()        # 反向传播
        optimizer.step()       # 每一步自动更新
        
        # 记录acc与loss
        train_acc  += (pred.argmax(1) == y).type(torch.float).sum().item()
        train_loss += loss.item()
            
    train_acc  /= size
    train_loss /= num_batches

    return train_acc, train_loss

In [15]:
def test (dataloader, model, loss_fn):
    size        = len(dataloader.dataset)  # 测试集的大小
    num_batches = len(dataloader)          # 批次数目, (size/batch_size，向上取整)
    test_loss, test_acc = 0, 0
    
    # 当不进行训练时，停止梯度更新，节省计算内存消耗
    with torch.no_grad():
        for imgs, target in dataloader:
            imgs, target = imgs.to(device), target.to(device)
            
            # 计算loss
            target_pred = model(imgs)
            loss        = loss_fn(target_pred, target)
            
            test_loss += loss.item()
            test_acc  += (target_pred.argmax(1) == target).type(torch.float).sum().item()

    test_acc  /= size
    test_loss /= num_batches

    return test_acc, test_loss

In [16]:
import copy

optimizer  = torch.optim.Adam(model.parameters(), lr= 1e-4)
loss_fn    = nn.CrossEntropyLoss() # 创建损失函数

epochs     = 10

train_loss = []
train_acc  = []
test_loss  = []
test_acc   = []

best_acc = 0    # 设置一个最佳准确率，作为最佳模型的判别指标

for epoch in range(epochs):
    
    model.train()
    epoch_train_acc, epoch_train_loss = train(train_dl, model, loss_fn, optimizer)
    
    model.eval()
    epoch_test_acc, epoch_test_loss = test(test_dl, model, loss_fn)
    
    # 保存最佳模型到 best_model
    if epoch_test_acc > best_acc:
        best_acc   = epoch_test_acc
        best_model = copy.deepcopy(model)
    
    train_acc.append(epoch_train_acc)
    train_loss.append(epoch_train_loss)
    test_acc.append(epoch_test_acc)
    test_loss.append(epoch_test_loss)
    
    # 获取当前的学习率
    lr = optimizer.state_dict()['param_groups'][0]['lr']
    
    template = ('Epoch:{:2d}, Train_acc:{:.1f}%, Train_loss:{:.3f}, Test_acc:{:.1f}%, Test_loss:{:.3f}, Lr:{:.2E}')
    print(template.format(epoch+1, epoch_train_acc*100, epoch_train_loss, 
                          epoch_test_acc*100, epoch_test_loss, lr))
    
# 保存最佳模型到文件中
PATH = './best_model.pth'  # 保存的参数文件名
torch.save(model.state_dict(), PATH)

print('Done')

Epoch: 1, Train_acc:42.5%, Train_loss:1.341, Test_acc:56.1%, Test_loss:1.162, Lr:1.00E-04
Epoch: 2, Train_acc:71.0%, Train_loss:0.845, Test_acc:57.9%, Test_loss:1.269, Lr:1.00E-04
Epoch: 3, Train_acc:70.1%, Train_loss:0.809, Test_acc:64.0%, Test_loss:1.011, Lr:1.00E-04
Epoch: 4, Train_acc:79.2%, Train_loss:0.627, Test_acc:74.6%, Test_loss:0.675, Lr:1.00E-04
Epoch: 5, Train_acc:77.4%, Train_loss:0.636, Test_acc:78.1%, Test_loss:0.741, Lr:1.00E-04
Epoch: 6, Train_acc:80.8%, Train_loss:0.539, Test_acc:75.4%, Test_loss:0.802, Lr:1.00E-04
Epoch: 7, Train_acc:79.9%, Train_loss:0.533, Test_acc:78.9%, Test_loss:0.889, Lr:1.00E-04
Epoch: 8, Train_acc:84.7%, Train_loss:0.414, Test_acc:80.7%, Test_loss:0.678, Lr:1.00E-04
Epoch: 9, Train_acc:89.6%, Train_loss:0.312, Test_acc:79.8%, Test_loss:0.656, Lr:1.00E-04
Epoch:10, Train_acc:89.6%, Train_loss:0.293, Test_acc:77.2%, Test_loss:0.745, Lr:1.00E-04
Done
